In [16]:
import os
import sys
import pandas as pd
import random

from sklearn.model_selection import GroupKFold

# If you intend to run the notebook from its original folder, example/start, you can keep the following line.
# Otherwise, you need to append the path to the leaspy folder
sys.path.append(os.path.join(os.getcwd(), '..', '..'))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from leaspy.main import Leaspy
from leaspy.inputs.data.data import Data
from leaspy.inputs.settings.algorithm_settings import AlgorithmSettings
from leaspy.utils.output.visualization.plotter import Plotter
from leaspy.inputs.data.result import Result

### Stability

In [30]:
df = pd.read_csv(os.path.join('_inputs', 'data.csv'), index_col= [0, 1])

indices = df.index.get_level_values(0).values
random.shuffle(indices)

if not os.path.isdir('_outputs/'):
    os.makedirs('_outputs/')

## Run on the whole  dataset first

In [ ]:
# Data
data = Data.from_dataframe(df)

if not os.path.isdir('_outputs/main'):
    os.makedirs('_outputs/main')

# Learn the model parameters
algo_settings = AlgorithmSettings('mcmc_saem', n_iter=2000)

leaspy = Leaspy("logistic_parallel")
leaspy.model.load_hyperparameters({'source_dimension': 2}) # Optional

leaspy.fit(data, algorithm_settings=algo_settings)
leaspy.save("_outputs/model_parameters.json") # optional


# Learn the personalization
settings = AlgorithmSettings('scipy_minimize')
results = leaspy.personalize(data, settings)

### Run on subsets of the dataset

- Learn the model prameters
- Learn the individual parameters

In [34]:
gkf = GroupKFold(n_splits=10)

settings_fit = AlgorithmSettings('mcmc_saem', n_iter=2000)
settings_personalization = AlgorithmSettings('scipy_minimize')

for i, (train, test) in enumerate(gkf.split(df, groups=indices)):
    df_train = df.iloc[train]
    df_test =  df.iloc[test]
    
    if not os.path.isdir(os.path.join('_outputs/', 'fold_'+str(i))):
        os.makedirs(os.path.join('_outputs', 'fold_'+str(i)))
    
    ### Need to have a newmodel every time to be sure to start from the same point
    leaspy = Leaspy("logistic_parallel")
    leaspy.model.load_hyperparameters({'source_dimension': 2}) # Optional
    
    ### Fit
    leaspy.fit(data, algorithm_settings=algo_settings)
    leaspy.save(os.path.join('_outputs', 'fold_'+str(i), 'model_parameters.json')) # optional
    
    # Personalize
    results = leaspy.personalize(leaspy_test, personalization_settings)
    ### SAVE INDIVIDUAL PARAMETERS IS NOT GONNA WORK
    leaspy.save_individual_parameters(os.path.join('_outputs', 'fold_'+str(i), 'individual_parameters.json'), results.individual_parameters)
                
                
    break

NameError: name 'data' is not defined

### Load the parameters and seen if it is it is always the same estimation

In [ ]:
main_model_parameters = leaspy.load()
main_individual_parameters = 

In [ ]:
cv_model_parameters = []
cv_individual_parameters= []

for i in range(10):
    path_model_parameters = os.path.join('_outputs', 'fold_'+str(i), 'model_parameters.json')
    leaspy.load(path)
    cv_model_parameters.append(leaspy.model.parameters)
    
    path_individual_parameters = os.path.join('_outputs', 'fold_'+str(i), 'model_parameters.json')
    i_p = leaspy.load_individual_parameters(path_individual_parameters)
    cv_individual_parameters.append(i_p)

In [ ]:
def display_model_parameters(main, cv):
    ### There is one result in the main, and n in the cv
    
def display_individual_parameters(main, cv):
    ### There is one individual parameters in the main, and one somewhere across the models